In [1]:
# import libraries
import numpy as np
import pandas as pd

In [4]:
# read data
data = pd.read_csv('../Datasets/Original Dataset/Dynamically Generated Hate Dataset v0.2.3.csv')

In [3]:
# print the first 5 rows of the data
print(data.head())

   Unnamed: 0    acl.id  X1  \
0           1      acl1   1   
1           2     acl10   2   
2           3    acl100   3   
3           4   acl1000   4   
4           5  acl10000   5   

                                                text label      type  \
0                                dalits are lowlives  hate  notgiven   
1             gay people are a burden to our society  hate  notgiven   
2                              Arabs are not welcome  hate  notgiven   
3  I'm not saying we should actually eliminate he...  hate  notgiven   
4                       bananas are for black people  hate  notgiven   

     target     level  split  round.base   annotator round acl.id.matched  
0  notgiven  original  train           1  GNZuCtwed3     1            NaN  
1  notgiven  original  train           1  GNZuCtwed3     1            NaN  
2  notgiven  original  train           1  vDe7GN0NrL     1            NaN  
3  notgiven  original  train           1  oemYWm1Tjg     1            NaN  


In [4]:
# displaying the fields and their data types
data.dtypes

Unnamed: 0         int64
acl.id            object
X1                 int64
text              object
label             object
type              object
target            object
level             object
split             object
round.base         int64
annotator         object
round             object
acl.id.matched    object
dtype: object

In [5]:
# finding the number of unique values in each field
data.nunique()

Unnamed: 0        41144
acl.id            41144
X1                41144
text              41134
label                 2
type                  7
target              409
level                 2
split                 3
round.base            4
annotator            20
round                 7
acl.id.matched    30098
dtype: int64

In [6]:
# displaying the number of unique values and their frequency in the fields label and type
data['label'].value_counts()

label
hate       22175
nothate    18969
Name: count, dtype: int64

In [7]:
data['type'].value_counts()

type
none              18969
derogation         9907
notgiven           7110
animosity          3439
dehumanization      906
threatening         606
support             207
Name: count, dtype: int64

In [8]:
data['split'].value_counts()

split
train    32924
test      4120
dev       4100
Name: count, dtype: int64

In [5]:
# There is a column called split which is used to split the data into training and testing data
# We will split the data into training and testing data, dev also is included in the training data since there is no option to mention dev data in the config files
train_data = data[data['split'] != 'test']
test_data = data[data['split'] == 'test']

In [11]:
# dropping the split column from the training and testing data
train_data = train_data.drop(columns=['split'])
test_data = test_data.drop(columns=['split'])

In [12]:
# save the training and testing data
train_data.to_csv('../Datasets/Split Datasets/Original/train_data.csv', index=False)
test_data.to_csv('../Datasets/Split Datasets/Original/test_data.csv', index=False)

In [13]:
# looking at the shape of the training data
print(train_data.shape)

(37024, 12)


In [14]:
# the only two columns we need for the classification task are text and label
train_data = train_data[['text', 'label']]
print(train_data.head())

                                                 text label
0                                 dalits are lowlives  hate
1              gay people are a burden to our society  hate
2                               Arabs are not welcome  hate
3   I'm not saying we should actually eliminate he...  hate
14  It's such a rape culture with these immigrants...  hate


In [15]:
# printing the number of unique labels in the data
print(train_data['label'].value_counts())

label
hate       19907
nothate    17117
Name: count, dtype: int64


In [18]:
# print 3 random rows from the data for each label
print(train_data[train_data['label'] == 'hate'].sample(3))
print(train_data[train_data['label'] == 'nothate'].sample(3))

                                                   text label
9914  I would love to sit down with pajeet people an...  hate
7347  Those Muslims are vile people, they do nothing...  hate
14    It's such a rape culture with these immigrants...  hate
                                                   text    label
5194  "Chinks belong in the zoo!"… doesn’t even make...  nothate
9566  contrary to popular opinion, I do not want to ...  nothate
1330  There are many Jewish people on our street but...  nothate


In [6]:
# now we will append a prompt to the text data to prepare incontext finetuning dataset
# The prompt is "Given the text, predict whether the text is hate speech or not, You can only output one word, either "hate" or "nothate" For example "dalits are lowlives" -> "hate"", "gay people are a burden to our society" -> "hate", "That chair is vile" -> "nothate", "Your hair is gross" -> "nothate", "Arabs are not welcome" -> "hate", "Obama is terrible" -> "nothate", Text:"
train_data['text'] = "Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate' For example dalits are lowlives -> hate, gay people are a burden to our society -> hate, That chair is vile -> nothate, Your hair is gross -> nothate, Arabs are not welcome -> hate, Obama is terrible -> nothate, Now the Text:" + train_data['text'] + " -> ?"

/tmp/ipykernel_915139/3706388296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['text'] = "Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate' For example dalits are lowlives -> hate, gay people are a burden to our society -> hate, That chair is vile -> nothate, Your hair is gross -> nothate, Arabs are not welcome -> hate, Obama is terrible -> nothate, Now the Text:" + train_data['text'] + " -> ?"


In [20]:
# now let us save the dataset as a json file with commas
train_data.to_json('../Datasets/Split Datasets/Classification/incontext/incontext_train_data.json', orient='records', lines=True)

In [26]:
# now let us create another dataset for the classification task without the examples, essentially a nocontext dataset
# load the training data
train_data = pd.read_csv('../Datasets/Split Datasets/Original/train_data.csv')

# the only two columns we need for the classification task are text and label
train_data = train_data[['text', 'label']]
print(train_data.head())

# Let us append the prompt to the text data which doesn't 
train_data['text'] = "Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate', the Text is:" + train_data['text'] + "-> ?"

# now let us save the dataset as a json file with commas
train_data.to_json('../Datasets/Split Datasets/Classification/nocontext/nocontext_train_data.json', orient='records', lines=True)


                                                text label
0                                dalits are lowlives  hate
1             gay people are a burden to our society  hate
2                              Arabs are not welcome  hate
3  I'm not saying we should actually eliminate he...  hate
4  It's such a rape culture with these immigrants...  hate


In [36]:
# let's load the tokenizer to check whether the dataset is in the correct format

from torchtune.models.llama3 import llama3_tokenizer

tokenizer = llama3_tokenizer("/tmp/Llama-3.2-1B-Instruct/original/tokenizer.model")

In [37]:
# let us see if we can load the data as an instruct_dataset for the incotext dataset

from torchtune.datasets import instruct_dataset

from torchtune.datasets import instruct_dataset
dataset = instruct_dataset(
     tokenizer=tokenizer,
     source="json",
     data_files="../Datasets/Split Datasets/Classification/incontext/incontext_train_data.json",
     column_map={
         "input": "text",
         "output": "label",
     },
     train_on_input=False,
     packed=False,
     split="train")



In [38]:
tokens = dataset[0]["tokens"]
tokenizer.decode(tokens)

"Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate' For example dalits are lowlives -> hate, gay people are a burden to our society -> hate, That chair is vile -> nothate, Your hair is gross -> nothate, Arabs are not welcome -> hate, Obama is terrible -> nothate, Now the Text:dalits are lowlives -> ?hate"

In [39]:
# let us see if we can load the data as an instruct_dataset for the nocontext dataset

from torchtune.datasets import instruct_dataset
dataset = instruct_dataset(
     tokenizer=tokenizer,
     source="json",
     data_files="../Datasets/Split Datasets/Classification/nocontext/nocontext_train_data.json",
     column_map={
         "input": "text",
         "output": "label",
     },
     train_on_input=False,
     packed=False,
     split="train")

In [40]:
tokens = dataset[0]["tokens"]
tokenizer.decode(tokens)

"Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate', the Text is:dalits are lowlives-> ?hate"

In [15]:
# now let us create the test data for the categorization task

# load the testing data
train_data = pd.read_csv('../Datasets/Split Datasets/Original/train_data.csv')

In [16]:
# the only two columns we need for the classification task are text and label
train_data = train_data[['text', 'type']]

In [ ]:
# A lot of people hate lazy foreigners for taking advantage of our social services, whilst sitting on their arse themselves contributing nothing - none
# n my view, the pro terrorist immigration rhetoric is an extreme concern. - derogation
# The concept of a nice person is the opposite to that of a woman - notgiven
# Yeah, refugees may not always have the choice of choosing the country they want to live in but they KNOW that some countries are already struggling, why would you come here? It's selfish and you can't blame people like me who barely make minimum wage to not want them here. It's truism and you know it. - Animosity
# BLM black monkeys are a disgrace to humanity - racists! - dehumanization
# Boris Johnson needs to stop being this crappy and kick all the immigrants out at last - threatening
# Murdering 6 million is not enough we would love to see more - support

In [17]:
# let us append the prompt to the test dataset
train_data['text'] = "Given the text, predict whether the text is hate speech or not and which category of hatespeech it falls under. Your output can only be one word, if the text is not hate speech, You output none. If you cannot decide in which category the text falls under, you can output notgiven. Otherwise you have to output one of the following categories derogation, animosity, dehumanization, threatening, support. For example, A lot of people hate lazy foreigners for taking advantage of our social services, whilst sitting on their arse themselves contributing nothing -> none, n my view, the pro terrorist immigration rhetoric is an extreme concern. -> derogation,The concept of a nice person is the opposite to that of a woman -> notgiven,Yeah, refugees may not always have the choice of choosing the country they want to live in but they KNOW that some countries are already struggling, why would you come here? It's selfish and you can't blame people like me who barely make minimum wage to not want them here. It's truism and you know it. -> Animosity,BLM black monkeys are a disgrace to humanity - racists! -> dehumanization,Boris Johnson needs to stop being this crappy and kick all the immigrants out at last -> threatening,Murdering 6 million is not enough we would love to see more -> support, Now The Text:" + train_data['text'] + "-> ?"

In [18]:
# now let us save the dataset as a json file with commas
train_data.to_json('../Datasets/Split Datasets/Categorization/incontext/incontext_train_data.json', orient='records', lines=True)

In [19]:
# load the testing data
train_data = pd.read_csv('../Datasets/Split Datasets/Original/train_data.csv')

In [20]:
# the only two columns we need for the classification task are text and label
train_data = train_data[['text', 'type']]

In [21]:
# let us append the prompt to the test dataset
train_data['text'] = "Given the text, predict whether the text is hate speech or not and which category of hatespeech it falls under. Your output can only be one word, if the text is not hate speech, You output none. If you cannot decide in which category the text falls under, you can output notgiven. Otherwise you have to output one of the following categories derogation, animosity, dehumanization, threatening, support. Now The Text:" + train_data['text'] + "-> ?"

In [22]:
# now let us save the dataset as a json file with commas
train_data.to_json('../Datasets/Split Datasets/Categorization/nocontext/nocontext_train_data.json', orient='records', lines=True)

In [ ]:
# let us prepare the dataset for testing for the classification task


In [2]:
# load the testing data
test_data = pd.read_csv('../Datasets/Split Datasets/Original/test_data.csv')

In [3]:
# the only two columns we need for the classification task are text and label
test_data = test_data[['text', 'label']]

In [4]:
# let us append the prompt to the test dataset
test_data['text'] = "Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate', the Text is:" + test_data['text'] + "-> ?"

In [5]:
# save the test data as a csv file
test_data.to_csv('../Datasets/Split Datasets/Classification/nocontext/nocontext_test_data.csv', index=False)

In [6]:
# repeating the same process for the incontext dataset
# load the testing data
test_data = pd.read_csv('../Datasets/Split Datasets/Original/test_data.csv')

# the only two columns we need for the classification task are text and label
test_data = test_data[['text', 'label']]

# let us append the prompt to the test dataset
test_data['text'] = "Given the text, predict whether the text is hate speech or not, You can only output one word, either 'hate' or 'nothate' For example dalits are lowlives -> hate, gay people are a burden to our society -> hate, That chair is vile -> nothate, Your hair is gross -> nothate, Arabs are not welcome -> hate, Obama is terrible -> nothate, Now the Text:" + test_data['text'] + "-> ?"

# save the test data as a csv file
test_data.to_csv('../Datasets/Split Datasets/Classification/incontext/incontext_test_data.csv', index=False)

In [7]:
# let us prepare the dataset for testing for the categorization task

# load the testing data
test_data = pd.read_csv('../Datasets/Split Datasets/Original/test_data.csv')

# the only two columns we need for the classification task are text and type
test_data = test_data[['text', 'type']]

# let us append the prompt to the test dataset
test_data['text'] = "Given the text, predict whether the text is hate speech or not and which category of hatespeech it falls under. Your output can only be one word, if the text is not hate speech, You output none. If you cannot decide in which category the text falls under, you can output notgiven. Otherwise you have to output one of the following categories derogation, animosity, dehumanization, threatening, support. Now The Text:" + test_data['text'] + "-> ?"

# save the test data as a csv file
test_data.to_csv('../Datasets/Split Datasets/Categorization/nocontext/nocontext_test_data.csv', index=False)

In [8]:
# repeating the same process for the incontext dataset
# load the testing data
test_data = pd.read_csv('../Datasets/Split Datasets/Original/test_data.csv')

# the only two columns we need for the classification task are text and type
test_data = test_data[['text', 'type']]

# let us append the prompt to the test dataset
test_data['text'] = "Given the text, predict whether the text is hate speech or not and which category of hatespeech it falls under. Your output can only be one word, if the text is not hate speech, You output none. If you cannot decide in which category the text falls under, you can output notgiven. Otherwise you have to output one of the following categories derogation, animosity, dehumanization, threatening, support. For example, A lot of people hate lazy foreigners for taking advantage of our social services, whilst sitting on their arse themselves contributing nothing -> none, n my view, the pro terrorist immigration rhetoric is an extreme concern. -> derogation,The concept of a nice person is the opposite to that of a woman -> notgiven,Yeah, refugees may not always have the choice of choosing the country they want to live in but they KNOW that some countries are already struggling, why would you come here? It's selfish and you can't blame people like me who barely make minimum wage to not want them here. It's truism and you know it. -> Animosity,BLM black monkeys are a disgrace to humanity - racists! -> dehumanization,Boris Johnson needs to stop being this crappy and kick all the immigrants out at last -> threatening,Murdering 6 million is not enough we would love to see more -> support, Now The Text:" + test_data['text'] + "-> ?"

# save the test data as a csv file
test_data.to_csv('../Datasets/Split Datasets/Categorization/incontext/incontext_test_data.csv', index=False)

In [2]:
# load the testing data
train_data = pd.read_csv('../Datasets/Split Datasets/Original/train_data.csv')

In [3]:
# the only two columns we need for the classification task are text and label
train_data = train_data[['text', 'type']]

In [4]:
# remove rows with type none and notgiven
train_data = train_data[train_data['type'] != 'none']
train_data = train_data[train_data['type'] != 'notgiven']


In [5]:
# let us append the prompt to the test dataset
train_data['text'] = "Given the text, Which is hate speech,predict which kind of hate speech the text falls under. Your output can only be one word.You can Output One of the following categories derogation, animosity, dehumanization, threatening, support. For example, n my view, the pro terrorist immigration rhetoric is an extreme concern. -> derogation,Yeah, refugees may not always have the choice of choosing the country they want to live in but they KNOW that some countries are already struggling, why would you come here? It's selfish and you can't blame people like me who barely make minimum wage to not want them here. It's truism and you know it. -> Animosity,BLM black monkeys are a disgrace to humanity - racists! -> dehumanization,Boris Johnson needs to stop being this crappy and kick all the immigrants out at last -> threatening,Murdering 6 million is not enough we would love to see more -> support, Now The Text:" + train_data['text'] + "-> ?"

In [6]:
# now let us save the dataset as a json file with commas
train_data.to_json('../Datasets/MISC/Given_Hate_Categorization_train_data.json', orient='records', lines=True)

In [7]:
# let's prepare the test set

# load the testing data
test_data = pd.read_csv('../Datasets/Split Datasets/Original/test_data.csv')

# the only two columns we need for the classification task are text and type
test_data = test_data[['text', 'type']]

# remove rows with type none and notgiven
test_data = test_data[test_data['type'] != 'none']
test_data = test_data[test_data['type'] != 'notgiven']

# let us append the prompt to the test dataset
test_data['text'] = "Given the text, Which is hate speech,predict which kind of hate speech the text falls under. Your output can only be one word.You can Output One of the following categories derogation, animosity, dehumanization, threatening, support. For example, n my view, the pro terrorist immigration rhetoric is an extreme concern. -> derogation,Yeah, refugees may not always have the choice of choosing the country they want to live in but they KNOW that some countries are already struggling, why would you come here? It's selfish and you can't blame people like me who barely make minimum wage to not want them here. It's truism and you know it. -> Animosity,BLM black monkeys are a disgrace to humanity - racists! -> dehumanization,Boris Johnson needs to stop being this crappy and kick all the immigrants out at last -> threatening,Murdering 6 million is not enough we would love to see more -> support, Now The Text:" + test_data['text'] + "-> ?"

# save the test data as a csv file
test_data.to_csv('../Datasets/MISC/Given_Hate_Categorization_test_data.csv', index=False)